#Long Form Question Answering &amp; Generated Content Detection

In this notebook, we are going to implement a long-form question answering and generated content detection model using natrual language processing (NLP) tools.

The dataset used in this project is a subset of [ELI5 dataset](https://huggingface.co/datasets/eli5_category).

The first step is to load the data and learn about the characteristics of data.
Then we have to do some pre-proceesing on the text data in order to make it ready to be given to a model to learn.

## install &amp; import libraries

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from datasets import load_dataset

## Load dataset and do some tasks on it

In [ ]:
dataset = load_dataset("eli5_category")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for eli5_category contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/eli5_category
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.war

Generating train split:   0%|          | 0/91772 [00:00<?, ? examples/s]

Generating validation1 split:   0%|          | 0/5446 [00:00<?, ? examples/s]

Generating validation2 split:   0%|          | 0/2375 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5411 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['q_id', 'title', 'selftext', 'category', 'subreddit', 'answers', 'title_urls', 'selftext_urls'],
        num_rows: 91772
    })
    validation1: Dataset({
        features: ['q_id', 'title', 'selftext', 'category', 'subreddit', 'answers', 'title_urls', 'selftext_urls'],
        num_rows: 5446
    })
    validation2: Dataset({
        features: ['q_id', 'title', 'selftext', 'category', 'subreddit', 'answers', 'title_urls', 'selftext_urls'],
        num_rows: 2375
    })
    test: Dataset({
        features: ['q_id', 'title', 'selftext', 'category', 'subreddit', 'answers', 'title_urls', 'selftext_urls'],
        num_rows: 5411
    })
})

In [ ]:
dataset['train'][0]

{'q_id': '5lchat',
 'title': "Why there was a 'leap second' added to the end of 2016?",
 'selftext': '',
 'category': 'Other',
 'subreddit': 'explainlikeimfive',
 'answers': {'a_id': ['dbuoyxl', 'dbur7gi', 'dbuotht'],
  'text': ['the rotation of the earth is not a constant. in fact the rotation of the earth is slowing down, which means that a full day is getting slightly longer. without leap seconds our clocks would slowly drift ever so slightly out of sync with the actual day. we could deal with this by redefining how how long 1 second is, making it slightly longer so that one day is still exactly 24*60*60 seconds. but in practice that is really inconvenient for a lot of our technology which relies on very precise timing. its easier to just move us ahead one second every couple of years or so.',
   "The Earth's rotation is not regular. It varies a bit, so sometimes we add a second. We do this to ensure that noon is always going to be sometime around mid-day. If we did not add leap sec

There is no useful information for selftext_urls.

In [ ]:
# Assuming dataset['train']['selftext_urls'] is your list
selftext_urls = dataset['train']['selftext_urls']

# Check if any element is different from the first element
is_different = any(url != selftext_urls[0] for url in selftext_urls)

if is_different:
    print("There are elements different from the first element in the list.")
else:
    print("All elements are the same as the first element in the list.")

All elements are the same as the first element in the list.


There is no useful information for title_urls.

In [ ]:
# Assuming dataset['train']['selftext_urls'] is your list
title_urls = dataset['train']['title_urls']

# Check if any element is different from the first element
is_different = any(url != title_urls[0] for url in title_urls)

if is_different:
    print("There are elements different from the first element in the list.")
else:
    print("All elements are the same as the first element in the list.")

All elements are the same as the first element in the list.


There is no useful information about subreddit.

In [ ]:
# Assuming dataset['train']['subreddit'] is your list
subreddit = dataset['train']['subreddit']

# Check if any element is different from the first element
is_different = any(url != subreddit[0] for url in subreddit)

if is_different:
    print("There are elements different from the first element in the list.")
else:
    print("All elements are the same as the first element in the list.")

All elements are the same as the first element in the list.


**There are some useful informations in selftext feature.**

In [ ]:
# Assuming dataset['train']['selftext'] is your list
selftext = dataset['train']['selftext']

# Check if any element is different from the first element
is_different = any(url != selftext[0] for url in selftext)

if is_different:
    print("There are elements different from the first element in the list.")
    # s = [url != selftext[0] for url in selftext]
    s1 = [i for i in range(len(selftext)) if selftext[i] != selftext[0]]
    print(s1)
else:
    print("All elements are the same as the first element in the list.")

There are elements different from the first element in the list.
[1, 2, 5, 8, 9, 10, 11, 14, 19, 25, 26, 27, 28, 31, 32, 33, 34, 36, 38, 41, 42, 44, 46, 48, 49, 51, 54, 55, 64, 66, 74, 76, 79, 82, 85, 91, 93, 95, 102, 113, 114, 115, 118, 119, 123, 124, 127, 128, 129, 133, 134, 139, 155, 156, 161, 162, 165, 166, 167, 168, 175, 176, 180, 184, 189, 191, 194, 197, 201, 202, 210, 211, 212, 214, 215, 216, 218, 219, 221, 222, 223, 226, 228, 229, 237, 238, 239, 241, 243, 246, 247, 248, 249, 250, 251, 253, 254, 255, 262, 263, 265, 269, 270, 271, 275, 276, 277, 282, 283, 286, 289, 291, 292, 296, 298, 299, 300, 303, 304, 306, 309, 311, 316, 318, 322, 323, 328, 331, 332, 335, 338, 347, 354, 356, 357, 359, 363, 365, 369, 372, 373, 374, 376, 377, 382, 384, 389, 392, 396, 401, 403, 405, 407, 409, 411, 413, 414, 415, 417, 420, 421, 425, 427, 428, 429, 431, 437, 438, 439, 441, 442, 447, 457, 458, 462, 469, 472, 474, 477, 482, 483, 484, 489, 490, 493, 494, 497, 498, 501, 503, 510, 512, 518, 519, 520, 52

## Simple Question Answering model

Now lets try to run a simple question answering model like BERT on this dataset.

The goal of this model is to generaqte the answer, not th elong answer for a question.

Store dataset in file

In [ ]:
# import os
# import json
# from datasets import load_dataset

# # Load the dataset
# dataset = load_dataset("eli5_category")

# # Directory where you want to save the JSON files
# output_dir = "dataset_json_files"

# # Create the directory if it doesn't exist
# os.makedirs(output_dir, exist_ok=True)

# # Iterate over each row in the dataset
# for i, example in enumerate(dataset["test"]):  # Change "train" to "validation" or "test" if needed
#     # Create a filename for the JSON file
#     filename = os.path.join(output_dir, f"example_{i}.json")

#     # Write the example as JSON to the file
#     with open(filename, "w") as f:
#         json.dump(example, f, indent=4)

#     print(f"Saved example {i} to {filename}")


Sample for Text to Speech

In [ ]:

# !pip install gTTS

# # Import the required module for text
# # to speech conversion
# from gtts import gTTS

# # This module is imported so that we can
# # play the converted audio
# import os

# # The text that you want to convert to audio
# mytext = 'Welcome to geeksforgeeks!'

# # Language in which you want to convert
# language = 'en'

# # Passing the text and language to the engine,
# # here we have marked slow=False. Which tells
# # the module that the converted audio should
# # have a high speed
# myobj = gTTS(text=mytext, lang=language, slow=False)

# # Saving the converted audio in a mp3 file named
# # welcome
# myobj.save("welcome.mp3")

# # Playing the converted file
# os.system("start welcome.mp3")


32512